In [3]:
from urdfpy import URDF
import math
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
import pandas as pd
import plotly
import plotly.graph_objs as go
import plotly.express as px
from scipy.spatial import distance

def get_angle(P,Q):
    R = np.dot(P,Q.T)
    theta = (np.trace(R) -1)/2
    if theta > 1:
        theta = 1
    elif theta < -1:
        theta = -1
    return np.arccos(theta) * (180/np.pi)


# index of end-effector is 17
end_effector = 17
acm1Frame = 16
acm1Fixed = 15
acm1aligned = 17
base_link = 0



robot = URDF.load('roombot_description/urdf/roombot.urdf')

#initial position:
m_init =[120, 180, -120]
offset_rad = (3*np.pi)/180

#m_init_rad = [(m_init[0]*np.pi)/180, (m_init[1]*np.pi)/180, (m_init[2]*np.pi)/180]
m_init_rad = [(m_init[0]*np.pi)/180, (m_init[1]*np.pi)/180, (m_init[2]*np.pi)/180]

m_mid_rad = [(10*np.pi)/180, 0, 0]
m_mid_rad2 = [(30*np.pi)/180, 0, 0]
m_mid_rad3 = [(50*np.pi)/180, 0, 0]
m_mid_rad4 = [(120*np.pi)/180, 180, -60]
m_mid_rad5 = [(120*np.pi)/180, 0, 0]
#m_mid_rad = [0, 0, 0]


# before move
fk = robot.link_fk()
r_1 = np.array(fk[robot.links[acm1aligned]]) 
r_b = np.array(fk[robot.links[base_link]])

config = {'M0': m_init_rad[0], 'M1': m_init_rad[1], 'M2': m_init_rad[2]}
fk = robot.link_fk(cfg=config)
r_1_in = np.array(fk[robot.links[acm1aligned]])
vect_in = r_1_in[:3,3]


config = {'M0': m_mid_rad[0], 'M1': m_mid_rad[1], 'M2': m_mid_rad[2]}
fk = robot.link_fk(cfg=config)
r_1_m = np.array(fk[robot.links[acm1aligned]])

config = {'M0': m_mid_rad2[0], 'M1': m_mid_rad2[1], 'M2': m_mid_rad2[2]}
fk = robot.link_fk(cfg=config)
#r_1_m2 = np.array(fk[robot.links[acm1aligned]])
r_1_m2 = np.array(fk[robot.links[acm1aligned]])


config = {'M0': m_mid_rad3[0], 'M1': m_mid_rad3[1], 'M2': m_mid_rad3[2]}
fk = robot.link_fk(cfg=config)
r_1_m3 = np.array(fk[robot.links[acm1aligned]])


config = {'M0': m_mid_rad4[0], 'M1': m_mid_rad4[1], 'M2': m_mid_rad4[2]}
fk = robot.link_fk(cfg=config)
r_1_m4 = np.array(fk[robot.links[acm1aligned]])

initial_pos_len = np.linalg.norm(r_1_in[:2, 3])
offset = offset_rad*initial_pos_len

theta = np.arctan(-r_1_in[:3, 3][0]/r_1_in[:3, 3][1])

x_true = [np.sin(theta+offset_rad)*initial_pos_len, -np.cos(theta+offset_rad)*initial_pos_len, r_1_in[:3, 3][2]]



print(theta)

0.0032813130802532182


In [4]:
r_1_in_shift = r_1_in.copy()
r_1_in_shift[:3,3] = x_true
vect_in_shift = r_1_in_shift[:3,3]

In [11]:
n_points = 40
thresh = 0.5
thresh2 = 2
bound = (20*np.pi)/180


m0_angles = np.linspace(-bound, bound, n_points)
m1_angles = np.linspace(-bound, bound, n_points)
m2_angles = np.linspace(-bound, bound, n_points)

point_of_interest = []
points_of_interest = []
pointsx = []
pointsy = []
pointsz = []
points = []
sol = []
hom_matrix = []
short_dist_all = []
short_dist = 100
sec_short_dist = 0
for m0 in m0_angles:
    for m1 in m1_angles:
        for m2 in m2_angles:
            old_angles = [m0, m1, m2]
            config = {'M0': m0+m_init_rad[0], 'M1': m1+m_init_rad[1], 'M2': m2+m_init_rad[2]} #reach random positons from initial motor command
            fk = robot.link_fk(cfg=config)
            r_2 = np.array(fk[robot.links[acm1aligned]])[:3, :3]
            vect = np.array(fk[robot.links[acm1aligned]])[:3, 3]
            
            if (get_angle(r_1_in_shift[:3, :3], r_2) < thresh): #if angle lower than 0.5°
                    
                pointsx.append(np.array(fk[robot.links[acm1aligned]])[0, 3])
                pointsy.append(np.array(fk[robot.links[acm1aligned]])[1, 3])
                pointsz.append(np.array(fk[robot.links[acm1aligned]])[2, 3]) 
                
                points.append(vect)
                sol.append([(m0+m_init_rad[0])*180/np.pi, m1+m_init_rad[1]*180/np.pi, m2+m_init_rad[2]*180/np.pi])
                norm = np.linalg.norm(vect_in_shift - vect)                                
             
                

In [12]:
#keep key positions:
reach_pos = pd.DataFrame(columns = ['x', 'y', 'z', 'x_shift', 'y_shift', 'z_shift'])
reach_pos['x'] = pointsx
reach_pos['y'] = pointsy
reach_pos['z'] = pointsz
results = pd.DataFrame(columns = ['vector', 'motor_command'])
results['vector'] = points
results['motor_command'] = sol

In [13]:
pointsx_shift =[]
pointsy_shift = []
pointsz_shift = []
pos_len = 0

for i in range(len(pointsx)):
    pos_len = np.linalg.norm([pointsx[i], pointsy[i]])
    theta = np.arctan(pointsx[i]/pointsy[i]) 
    
    pointsx_shift.append(np.sin(offset_rad-theta)*initial_pos_len)
    pointsy_shift.append(-np.cos(offset_rad-theta)*initial_pos_len)
    pointsz_shift.append(pointsz[i])
    


reach_pos['x_shift'] = pointsx_shift
reach_pos['y_shift'] = pointsy_shift
reach_pos['z_shift'] = pointsz_shift


for i in range(len(results['motor_command'])):
    if (results['motor_command'][i][0] < 120):
        reach_pos = reach_pos.drop(i)
        results = results.drop(i)
        

In [14]:
import math

def dotproduct(v1, v2):
    return sum((a*b) for a, b in zip(v1, v2))

def length(v):
    return math.sqrt(dotproduct(v, v))

def angle(v1, v2):
    return math.acos(dotproduct(v1, v2) / (length(v1) * length(v2)))

angle = angle([r_1_in_shift[:3,3][0],r_1_in_shift[:3,3][1],r_1_in_shift[:3,3][2]], [r_1_in[:3, 3][0], r_1_in[:3, 3][1], r_1_in[:3, 3][2]])

In [15]:
#display results

sleep_pos = go.Scatter3d(
    x=[r_1[:3, 3][0],r_b[:3, 3][0]],
    y=[r_1[:3,3][1], r_b[:3,3][1]],
    z =[r_1[:3,3][2], r_b[:3,3][2]],
    name="sleep position"
)
true_pos = go.Scatter3d(
    x=[r_b[:3, 3][0], r_1_in_shift[:3,3][0]],
    y=[r_b[:3,3][1], r_1_in_shift[:3,3][1]],
    z = [r_b[:3,3][2], r_1_in_shift[:3,3][2]],
    name="initial position"
)

initial_pos = go.Scatter3d(
    x=[r_b[:3, 3][0], r_1_in[:3, 3][0]],
    y=[r_b[:3,3][1], r_1_in[:3, 3][1]],
    z = [r_b[:3,3][2], r_1_in[:3, 3][2]],
    name="target position"
)


reachable_pos = go.Scatter3d(
    x=reach_pos['x'],
    y=reach_pos['y'],
    z = reach_pos['z'],
    name="key positions"
)

shift_pos = go.Scatter3d(
    x=reach_pos['x_shift'] ,
    y=reach_pos['y_shift'] ,
    z = reach_pos['z_shift'] ,
    name="key positions"
)


mid_pos = go.Scatter3d(
    x=[r_b[:3, 3][0], r_1_m[:3, 3][0]],
    y=[r_b[:3,3][1], r_1_m[:3, 3][1]],
    z = [r_b[:3,3][2], r_1_m[:3, 3][2]],
    name="mid position"
)

mid_pos2 = go.Scatter3d(
    x=[r_b[:3, 3][0], r_1_m2[:3, 3][0]],
    y=[r_b[:3,3][1], r_1_m2[:3, 3][1]],
    z = [r_b[:3,3][2], r_1_m2[:3, 3][2]],
    name="mid position2"
)

mid_pos3 = go.Scatter3d(
    x=[r_b[:3, 3][0], r_1_m3[:3, 3][0]],
    y=[r_b[:3,3][1], r_1_m3[:3, 3][1]],
    z = [r_b[:3,3][2], r_1_m3[:3, 3][2]],
    name="mid position3"
)


data = [initial_pos, sleep_pos, true_pos, shift_pos]

fig = go.Figure(data=data)
fig.show()